In [1]:
!pip3 install pyspark

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import os
os.environ['PYSPARK_PYTHON'] = 'python'

In [3]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName("importing and data slicing").getOrCreate() # name should be quniue for less conflicts

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/26 19:38:13 WARN Utils: Your hostname, Biplovs-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.66 instead (on interface en0)
25/11/26 19:38:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/26 19:38:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/26 19:38:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [17]:
flights = spark.read.csv('./nyc/nyc_flights.csv', header = True)
airlines = spark.read.csv('./nyc/nyc_airlines.csv', header = True)
airports = spark.read.csv('./nyc/nyc_airports.csv', header = True)

In [18]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [21]:
flights.createOrReplaceTempView('flights')
airlines.createOrReplaceTempView('airlines')
airports.createOrReplaceTempView('airports')


In [22]:
query = """ SELECT * FROM flights """
result = spark.sql(query)
result.show()


+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+--------------------+
|year|month|day|dep_time|sched_dep_time|dep_delay|arr_time|sched_arr_time|arr_delay|carrier|flight|tailnum|origin|dest|air_time|distance|hour|minute|           time_hour|
+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+--------------------+
|2013|    1|  1|     517|           515|        2|     830|           819|       11|     UA|  1545| N14228|   EWR| IAH|     227|    1400|   5|    15|2013-01-01T10:00:00Z|
|2013|    1|  1|     533|           529|        4|     850|           830|       20|     UA|  1714| N24211|   LGA| IAH|     227|    1416|   5|    29|2013-01-01T10:00:00Z|
|2013|    1|  1|     542|           540|        2|     923|           850|       33|     AA|  1141| N619AA|   JFK| MIA|     160|    1089|   5|   

In [34]:
# challange 1: which airline has the highest number of delayed departures
# query1 = """SELECT a.name,
# COUNT(*) as num_delay FROM flights f
# LEFT JOIN airlines a ON f.carrier = a.carrier
# WHERE f.dep_delay > 0 
# GROUP BY a.name
# ORDER BY num_delay DESC
# LIMIT 1"""


# Check your Python code to ensure query1 is defined as:
query1 = """
SELECT 
    a.name,
    COUNT(*) AS num_delay
FROM 
    flights f
LEFT JOIN 
    airlines a 
ON 
    f.carrier = a.carrier
WHERE 
    try_cast(f.dep_delay AS INT) > 0 -- <-- Use try_cast to handle 'NA'
GROUP BY 
    a.name
ORDER BY 
    num_delay DESC
LIMIT 1
"""
result1 = spark.sql(query1)
result1.show()

[Stage 17:>                                                         (0 + 8) / 8]

+--------------------+---------+
|                name|num_delay|
+--------------------+---------+
|United Air Lines ...|    27261|
+--------------------+---------+



In [35]:
# challange 2: fastest araival in which airport
query2 = """SELECT a.name, 
AVG(arr_delay) as mean_delay 
FROM flights f JOIN aiports on f.dest = a.faa
"""
result2 = spark.sql(query1)
result2.show()

[Stage 21:>                                                         (0 + 8) / 8]

+--------------------+---------+
|                name|num_delay|
+--------------------+---------+
|United Air Lines ...|    27261|
+--------------------+---------+



In [ ]:
# challange 3 : jfk to sea aiport samma avg maximum and minimum airtime
query3 = 

In [ ]:
flights.printSchema()